<a href="https://colab.research.google.com/github/Hagga-wagga1/SOLANACRYPTOBOT/blob/main/SolanaBotten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ccxt numpy python-dotenv

In [ ]:
import os
import time
import threading
import ccxt
import numpy as np
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

# Configuration
SYMBOL = "SOL/USD"
INITIAL_INVESTMENT = 1000
MAX_GRIDS = 80
SAFETY_DRAWDOWN = 2.0
TAKE_PROFIT = 3.0
COOLDOWN = 300  # 5 minutes in seconds

class SolanaGridBot:
    def __init__(self):
        self.running = False
        self.exchange = ccxt.cryptocom({
            'apiKey': os.getenv('CRO_API_KEY'),
            'secret': os.getenv('CRO_API_SECRET'),
            'enableRateLimit': True,
        })
        self.current_price = 0.0
        self.grids = []
        self.lock = threading.Lock()
        self.stop_event = threading.Event()

    # ----------------------
    # Core Trading Logic
    # ----------------------

    def calculate_volatility(self):
        ohlcv = self.exchange.fetch_ohlcv(SYMBOL, '5m', limit=100)
        highs = np.array([x[2] for x in ohlcv])
        lows = np.array([x[3] for x in ohlcv])
        return np.mean(highs - lows) / self.current_price * 100

    def update_grids(self):
        self.current_price = self.exchange.fetch_ticker(SYMBOL)['last']
        volatility = self.calculate_volatility()
        grid_count = min(MAX_GRIDS, int(30 + (volatility * 2)))
        price_range = self.current_price * (volatility * 1.5) / 100
        upper = self.current_price + price_range
        lower = self.current_price - price_range
        self.grids = np.linspace(lower, upper, grid_count).round(3)

    def place_orders(self):
        self.exchange.cancel_all_orders(SYMBOL)
        usd_per_grid = INITIAL_INVESTMENT / len(self.grids)

        for price in self.grids:
            try:
                # Buy order
                buy_price = round(price * 0.999, 3)
                amount = usd_per_grid / buy_price
                self.exchange.create_limit_buy_order(SYMBOL, amount, buy_price)

                # Sell order
                sell_price = round(price * 1.006, 3)
                self.exchange.create_limit_sell_order(SYMBOL, amount, sell_price)
            except Exception as e:
                print(f"Order error: {e}")

    # ----------------------
    # Safety & Monitoring
    # ----------------------

    def safety_monitor(self):
        while not self.stop_event.is_set():
            try:
                balance = self.exchange.fetch_balance()
                current = self.current_price

                if current < (self.grids[0] * 0.98):
                    print("⚠️ Emergency sell triggered!")
                    self.exchange.create_market_sell_order(SYMBOL, balance['SOL']['free'])
                    self.stop_event.set()
                    self.restart_bot()

                time.sleep(10)
            except Exception as e:
                print(f"Safety error: {e}")

    def restart_bot(self):
        print(f"⏳ Restarting in {COOLDOWN//60} minutes...")
        time.sleep(COOLDOWN)
        self.__init__()
        self.start_bot()

    # ----------------------
    # User Controls
    # ----------------------

    def start_bot(self):
        if self.running: return

        self.running = True
        self.stop_event.clear()

        print("\n🟢 Bot started")
        print("Press 'S' to stop | 'R' to reload grids")

        # Start threads
        threads = [
            threading.Thread(target=self.grid_loop),
            threading.Thread(target=self.safety_monitor),
            threading.Thread(target=self.command_listener)
        ]

        for t in threads:
            t.start()

    def stop_bot(self):
        self.running = False
        self.stop_event.set()
        print("\n🔴 Bot stopped")

    def command_listener(self):
        while self.running:
            cmd = input().lower()
            if cmd == 's':
                self.stop_bot()
            elif cmd == 'r':
                with self.lock:
                    self.update_grids()
                    self.place_orders()
                print("♻️ Grids reloaded")

    # ----------------------
    # Main Loop
    # ----------------------

    def grid_loop(self):
        while self.running:
            try:
                with self.lock:
                    self.update_grids()
                    self.place_orders()
                    print(f"📊 Active grids: {len(self.grids)}")
                time.sleep(60)
            except Exception as e:
                print(f"Grid error: {e}")
                self.stop_bot()

if __name__ == "__main__":
    bot = SolanaGridBot()

    print("""
    ======================
    Crypto.com Grid Bot
    ======================
    Commands:
    - Enter: Start bot
    - S: Stop bot
    - R: Reload grids
    - Ctrl+C: Exit
    """)

    input("Press Enter to start...")

    try:
        bot.start_bot()
        while bot.running:
            time.sleep(1)
    except KeyboardInterrupt:
        bot.stop_bot()


    Crypto.com Grid Bot
    Commands:
    - Enter: Start bot
    - S: Stop bot
    - R: Reload grids
    - Ctrl+C: Exit
    


KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import time
import threading
import ccxt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import datetime, timedelta

load_dotenv()

# Configuration
SYMBOL = "SOL/USD"
INITIAL_INVESTMENT = 1000
MAX_GRIDS = 80
SAFETY_DRAWDOWN = 2.0
LOOKBACK_DAYS = 90  # For backtesting and forecasting

class AdvancedGridBot:
    def __init__(self):
        self.active = False
        self.exchange = ccxt.cryptocom({
            'apiKey': os.getenv('CRO_API_KEY'),
            'secret': os.getenv('CRO_API_SECRET'),
            'enableRateLimit': True,
        })
        self.historical_data = None
        self.forecast_model = None
        self.lock = threading.Lock()

    # ----------------------
    # Core Trading Engine
    # ----------------------

    def calculate_optimal_grids(self, prices):
        """Vectorized grid optimization using historical volatility"""
        returns = np.diff(np.log(prices))
        volatility = np.std(returns) * np.sqrt(365)  # Annualized volatility

        # Dynamic grid calculation
        grid_levels = min(MAX_GRIDS, int(volatility * 20))  # 5% volatility = 40 grids
        price_range = prices[-1] * volatility * 2  # 2σ range
        return np.linspace(prices[-1] - price_range/2,
                          prices[-1] + price_range/2,
                          grid_levels).round(3)

    def vectorized_backtest(self, grids):
        """Fast backtesting using NumPy vectorization"""
        entry_points = grids[:-1]
        exit_points = grids[1:]

        # Simulate trades across historical data
        positions = np.zeros(len(self.historical_data))
        for idx, price in enumerate(self.historical_data['close']):
            in_grid = np.where((price > entry_points) & (price < exit_points))[0]
            if in_grid.size > 0:
                positions[idx] = 1 / len(in_grid)  # Position sizing

        # Calculate returns
        returns = np.diff(np.log(self.historical_data['close'])) * positions[:-1]
        return np.cumsum(returns)[-1]  # Total log return

    # ----------------------
    # Predictive Analytics
    # ----------------------

    def train_forecast_model(self):
        """Polynomial regression price forecasting"""
        X = np.arange(len(self.historical_data)).reshape(-1, 1)
        y = self.historical_data['close'].values

        poly = PolynomialFeatures(degree=2)
        X_poly = poly.fit_transform(X)

        self.forecast_model = LinearRegression()
        self.forecast_model.fit(X_poly, y)

    def predict_next_day(self):
        """Forecast next 24 hours using trained model"""
        if not self.forecast_model:
            return None

        last_idx = len(self.historical_data)
        next_X = poly.fit_transform([[last_idx + 24]])  # 24 hours ahead
        return self.forecast_model.predict(next_X)[0]

    # ----------------------
    # Live Trading System
    # ----------------------

    def live_trading_loop(self):
        """Optimized live trading with predictive elements"""
        while self.active:
            current_price = self.exchange.fetch_ticker(SYMBOL)['last']

            # Update grids every 15 minutes with forecast
            if datetime.now().minute % 15 == 0:
                forecast = self.predict_next_day()
                grids = self.calculate_optimal_grids(
                    np.append(self.historical_data['close'], [current_price, forecast])

                with self.lock:
                    self.adjust_orders(grids)

            time.sleep(60)

    # ----------------------
    # User Interface
    # ----------------------

    def show_analytics_dashboard(self):
        """Command-line analytics display"""
        while True:
            os.system('cls' if os.name == 'nt' else 'clear')
            print(f"""
            === SOL/USD Grid Bot Analytics ===
            [1] Run Backtest (Last {LOOKBACK_DAYS} Days)
            [2] Live Trading Mode
            [3] View Price Forecast
            [4] Return to Menu
            """)

            choice = input("Select option: ")
            if choice == '1':
                self.run_complete_backtest()
            elif choice == '2':
                self.live_control_menu()
            elif choice == '3':
                self.show_forecast()
            elif choice == '4':
                break

    def run_complete_backtest(self):
        """Full historical simulation with optimization"""
        self.load_historical_data()
        best_performance = -np.inf
        best_grids = []

        # Grid search optimization
        for _ in range(100):
            test_grids = self.calculate_optimal_grids(self.historical_data['close'])
            performance = self.vectorized_backtest(test_grids)

            if performance > best_performance:
                best_performance = performance
                best_grids = test_grids

        print(f"\n🔥 Optimal Grids: {best_grids[:5]}...{best_grids[-5:]}")
        print(f"📈 Projected Annual Return: {best_performance * 365:.1f}%")

    # ----------------------
    # Data Management
    # ----------------------

    def load_historical_data(self):
        """Load OHLCV data from exchange"""
        since = self.exchange.parse8601((datetime.now() -
            timedelta(days=LOOKBACK_DAYS)).isoformat())

        self.historical_data = self.exchange.fetch_ohlcv(
            SYMBOL, '1h', since=since, limit=1000)
        self.historical_data = pd.DataFrame(
            self.historical_data,
            columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
        )
        self.train_forecast_model()

if __name__ == "__main__":
    bot = AdvancedGridBot()
    bot.load_historical_data()

    while True:
        os.system('cls' if os.name == 'nt' else 'clear')
        print(f"""
        === AI-Powered Grid Trading System ===
        Last Price: {bot.historical_data['close'].iloc[-1]:.2f}
        24h Forecast: {bot.predict_next_day():.2f}
        ------------------------------
        [1] Analytics Dashboard
        [2] Start Live Trading
        [3] Exit
        """)

        choice = input("Select option: ")
        if choice == '1':
            bot.show_analytics_dashboard()
        elif choice == '2':
            bot.live_control_menu()
        elif choice == '3':
            exit()

In [ ]:
pip install ccxt numpy pandas scikit-learn python-dotenv

In [ ]:
======================
Crypto.com Grid Bot
======================
1. Start Trading Bot
2. Check Balance
3. Exit

In [ ]:
# Step 5: Modified Main Bot Code
class ColabGridBot:
    def __init__(self):
        self.exchange = ccxt.cryptocom({
            'apiKey': os.getenv('CRO_API_KEY'),
            'secret': os.getenv('CRO_API_SECRET'),
            'enableRateLimit': True,
        })
        self.running = False
        self.grids = []
        self.pending_buys = []
        self.pending_sells = []

    def prepare_orders(self, current_price):
        """Generate proposed orders without executing

In [ ]:
!python -m pip install Django==5.1.6

In [ ]:
pip install django matplotlib plotly pandas

In [ ]:
# trades/models.py
from django.db import models

class Trade(models.Model):
    timestamp = models.DateTimeField(auto_now_add=True)
    symbol = models.CharField(max_length=10)
    price = models.FloatField()
    quantity = models.FloatField()
    side = models.CharField(max_length=4)  # BUY/SELL
    profit = models.FloatField(null=True)

In [ ]:
# trades/views.py
import plotly.express as px
from django.shortcuts import render

def interactive_chart(request):
    trades = Trade.objects.all()
    df = pd.DataFrame.from_records(trades.values())

    fig = px.line(df, x='timestamp', y='profit',
                 title="Trading Performance")
    chart = fig.to_html(full_html=False)

    return render(request, 'interactive_chart.html', {'chart': chart})

In [ ]:
<!-- trades/templates/chart.html -->
<img src="data:image/png;base64,{{ chart }}" alt="Performance Chart">

In [ ]:
<!-- trades/templates/interactive_chart.html -->
{% autoescape off %}
{{ chart }}
{% endautoescape %}

In [ ]:
# trades/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('performance/', views.performance_chart),
    path('interactive/', views.interactive_chart),
]

In [ ]:
# trades/views.py
def dashboard(request):
    # Price chart
    price_fig = px.line(price_data, x='time', y='price')
    price_chart = price_fig.to_html()

    # Performance chart
    perf_fig = px.area(performance_data, x='date', y='equity')
    perf_chart = perf_fig.to_html()

    return render(request, 'dashboard.html', {
        'price_chart': price_chart,
        'perf_chart': perf_chart
    })

In [ ]:
# consumers.py
from channels.generic.websocket import AsyncWebsocketConsumer
import json

class ChartConsumer(AsyncWebsocketConsumer):
    async def connect(self):
        await self.accept()

    async def send_update(self, event):
        await self.send(text_data=json.dumps(event['data']))

In [ ]:
python manage.py makemigrations
python manage.py migrate
python manage.py runserver

In [ ]:
import time
import ccxt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from dotenv import load_dotenv

load_dotenv()

class VisualTradingBot:
    def __init__(self):
        self.exchange = ccxt.binance({
            'apiKey': os.getenv('API_KEY'),
            'secret': os.getenv('API_SECRET'),
            'enableRateLimit': True
        })
        self.history = pd.DataFrame(columns=['timestamp', 'price', 'action'])
        self.fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                vertical_spacing=0.05,
                                row_heights=[0.7, 0.3])

    def update_chart(self):
        """Update the interactive price chart with annotations"""
        clear_output(wait=True)

        # Create candlestick chart
        ohlcv = self.exchange.fetch_ohlcv('BTC/USDT', '15m', limit=50)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

        self.fig = go.Figure()
        self.fig.add_trace(go.Candlestick(x=df['timestamp'],
                            open=df['open'],
                            high=df['high'],
                            low=df['low'],
                            close=df['close'],
                            name='Price'))

        # Add trading signals
        trades = self.history[self.history['action'].isin(['BUY', 'SELL'])]
        self.fig.add_trace(go.Scatter(
            x=trades['timestamp'],
            y=trades['price'],
            mode='markers',
            marker=dict(
                color=['green' if a == 'BUY' else 'red' for a in trades['action']],
                size=10
            ),
            name='Trades'
        ))

        # Add indicators
        self.fig.add_trace(go.Scatter(
            x=df['timestamp'],
            y=df['close'].rolling(20).mean(),
            line=dict(color='orange', width=2),
            name='20-period MA'
        ), row=1, col=1)

        # Show plot
        self.fig.show()

    def make_trade_decision(self, price):
        """Example trading strategy with visual feedback"""
        ma = pd.Series(self.exchange.fetch_ohlcv('BTC/USDT', '15m', limit=20))[4].mean()

        if price > ma * 1.02:
            action = 'SELL'
            marker_color = 'red'
        elif price < ma * 0.98:
            action = 'BUY'
            marker_color = 'green'
        else:
            action = 'HOLD'

        # Add to history
        self.history.loc[len(self.history)] = {
            'timestamp': pd.Timestamp.now(),
            'price': price,
            'action': action
        }

        return action

    def run(self):
        print("📈 Starting Visual Trading Bot")
        try:
            while True:
                # Get market data
                ticker = self.exchange.fetch_ticker('BTC/USDT')
                price = ticker['last']

                # Make trading decision
                action = self.make_trade_decision(price)

                # Update display
                self.update_chart()
                print(f"\nLast Price: ${price:.2f}")
                print(f"Current Action: {action}")
                print(f"Trade Count: {len(self.history)}")

                # Simulate order placement
                if action in ['BUY', 'SELL']:
                    print(f"Placing {action} order...")

                time.sleep(30)  # Update every 30 seconds

        except KeyboardInterrupt:
            print("\n🔴 Bot Stopped")

if __name__ == "__main__":
    bot = VisualTradingBot()
    bot.run()

In [ ]:
pip install ccxt plotly pandas ipython python-dotenv

In [ ]:
# Relative Strength Index
delta = df['close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
rs = gain / loss
rsi = 100 - (100 / (1 + rs))

self.fig.add_trace(go.Scatter(x=df['timestamp'], y=rsi,
                           line=dict(color='purple'), row=2, col=1)

In [ ]:
def show_order_book(self):
    order_book = self.exchange.fetch_order_book('SOL/USD')
    bids = pd.DataFrame(order_book['bids'], columns=['price', 'amount'])
    asks = pd.DataFrame(order_book['asks'], columns=['price', 'amount'])

    display(go.Figure()
        .add_trace(go.Bar(x=bids['price'], y=bids['amount'], name='Bids'))
        .add_trace(go.Bar(x=asks['price'], y=asks['amount'], name='Asks'))
        .update_layout(title='Order Book Depth')))

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
pip install streamlit

In [ ]:
streamlit hello

In [ ]:
import time
import threading
import ccxt
import pandas as pd
import plotly.graph_objects as go
import streamlit as st
from dotenv import load_dotenv

load_dotenv()

class VisualTradingApp:
    def __init__(self):
        self.exchange = ccxt.binance({
            'apiKey': os.getenv('API_KEY'),
            'secret': os.getenv('API_SECRET'),
            'enableRateLimit': True
        })
        self.running = False
        self.history = pd.DataFrame()
        self.lock = threading.Lock()

    def get_market_data(self, symbol='BTC/USDT'):
        """Fetch OHLCV data with support/resistance levels"""
        ohlcv = self.exchange.fetch_ohlcv(symbol, '15m', limit=100)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['support'] = df['low'].rolling(20).min()
        df['resistance'] = df['high'].rolling(20).max()
        return df

    def trading_strategy(self, symbol):
        """Combined grid + support strategy"""
        while self.running:
            try:
                with self.lock:
                    df = self.get_market_data(symbol)
                    current_price = self.exchange.fetch_ticker(symbol)['last']

                    # Trading logic
                    if current_price > df['resistance'].iloc[-1]:
                        self.place_order(symbol, 'sell', current_price)
                    elif current_price < df['support'].iloc[-1]:
                        self.place_order(symbol, 'buy', current_price)

                    time.sleep(10)

            except Exception as e:
                st.error(f"Error: {str(e)}")
                time.sleep(5)

    def place_order(self, symbol, side, price):
        """Execute and log orders"""
        try:
            order = self.exchange.create_order(
                symbol=symbol,
                type='limit',
                side=side,
                amount=0.001,
                price=price
            )
            self.history = pd.concat([
                self.history,
                pd.DataFrame([{
                    'timestamp': pd.Timestamp.now(),
                    'price': price,
                    'side': side,
                    'status': 'open'
                }])
            ])

        except ccxt.BaseError as e:
            st.warning(f"Order failed: {str(e)}")

    def update_dashboard(self):
        """Real-time dashboard components"""
        st.title("💰 Cryptocurrency Trading Dashboard")

        # Strategy Controls
        col1, col2 = st.columns(2)
        with col1:
            if st.button("Start Trading") and not self.running:
                self.running = True
                threading.Thread(target=self.trading_strategy, args=('BTC/USDT',)).start()
        with col2:
            if st.button("Stop Trading"):
                self.running = False
                self.exchange.cancel_all_orders('BTC/USDT')

        # Market Data Visualization
        df = self.get_market_data()
        fig = go.Figure()
        fig.add_trace(go.Candlestick(
            x=df['timestamp'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Price'
        ))
        fig.update_layout(height=600)
        st.plotly_chart(fig, use_container_width=True)

        # Trading History
        st.subheader("Order History")
        st.dataframe(self.history.tail(10))

        # Account Summary
        balance = self.exchange.fetch_balance()
        st.metric(label="BTC Balance", value=f"{balance['BTC']['free']:.4f}")
        st.metric(label="USDT Balance", value=f"{balance['USDT']['free']:.2f}")

if __name__ == "__main__":
    app = VisualTradingApp()
    app.update_dashboard()

In [ ]:
streamlit==1.22.0
plotly==5.13.0
pandas==2.0.3
ccxt==4.1.52
python-dotenv==1.0.0

In [ ]:
# Install dependencies
pip install -r requirements.txt

# Run the visual program
streamlit run app.py

In [ ]:
# Add moving average
fig.add_trace(go.Scatter(
    x=df['timestamp'],
    y=df['close'].rolling(20).mean(),
    line=dict(color='orange'),
    name='20-MA'
))

In [ ]:
def trading_strategy(self, symbol):
    while self.running:
        # Add your custom logic here
        if rsi < 30:  # Example condition
            self.place_order(symbol, 'buy', current_price)

In [ ]:
# Add to place_order method
st.toast(f"Order placed: {side} @ {price}", icon="✅")

In [ ]:
!python bot.py